In [199]:
import json
import random
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from tqdm import trange

from pprint import pprint

from sklearn.feature_extraction.text import TfidfVectorizer

import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F

from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    BertModel, RobertaModel,
    BertPreTrainedModel,
    AdamW, get_linear_schedule_with_warmup,
    TrainingArguments,
)
from datasets import (
    Dataset,
    load_from_disk,
    concatenate_datasets,
)

from typing import List
from torch.utils.data import Sampler

In [200]:
# 난수 고정
def set_seed(random_seed):
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)  # if use multi-GPU
    random.seed(random_seed)
    np.random.seed(random_seed)
    
set_seed(42) # magic number :)

In [201]:
print ("PyTorch version:[%s]."%(torch.__version__))
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print ("device:[%s]."%(device))

PyTorch version:[1.7.1].
device:[cuda:0].


## Validation

In [5]:
dataset = load_from_disk('/opt/ml/data/train_dataset')
train_dataset = dataset['train']

In [6]:
data = pd.read_csv('/opt/ml/custom/top100_wikipedia.csv')

In [7]:
doc_indices = []
doc_scores = []
for i in range(len(data)) :
    tmp_indices = eval(data['document_id'][i])
    tmp_scores = eval(data['score'][i])
    doc_indices.append(tmp_indices)
    doc_scores.append(tmp_scores)

In [8]:
with open('/opt/ml/data/wikipedia_documents.json', "r", encoding="utf-8") as f:
    wiki = json.load(f)

corpus = []
for v in wiki.values() :
    corpus.append(v['text'])

In [9]:
class BertEncoder(BertPreTrainedModel):
    def __init__(self, config):
        super(BertEncoder, self).__init__(config)

        self.bert = BertModel(config)
        self.init_weights()
        classifier_dropout=(
            config.classifier_dropout if config.classifier_dropout is not None else config.hidden_dropout_prob
        )
        self.dropout = torch.nn.Dropout(classifier_dropout)
        self.linear = torch.nn.Linear(config.hidden_size, 1)
      
    def forward(
            self,
            input_ids, 
            attention_mask=None,
            token_type_ids=None
        ): 

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        
        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output)
        output = self.linear(pooled_output)
        return output

In [10]:
c_encoder = torch.load('/opt/ml/custom/c_encoder_e40_b16.pt')
model_checkpoint = "klue/bert-base"
# 혹시 위에서 사용한 encoder가 있다면 주석처리 후 진행해주세요 (CUDA ...)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [11]:
question_data = dataset['validation']['question']
with torch.no_grad() : 
    c_encoder.eval()

    result_scores = []
    result_indices = []
    for i in tqdm(range(len(question_data))) :
        question = question_data[i]
        question_score = []
        for indice in doc_indices[i] :
            passage = corpus[indice]
            tokenized_examples = tokenizer(
                question,
                passage,
                truncation="only_second",
                max_length=512,
                stride=128,
                return_overflowing_tokens=True,
                return_offsets_mapping=True,
                #return_token_type_ids=False,  # roberta모델을 사용할 경우 False, bert를 사용할 경우 True로 표기해야합니다.
                padding="max_length",
                return_tensors='pt'
            )
            score = 0
            for i in range(len(tokenized_examples['input_ids'])) :
                c_input = {
                    'input_ids' : torch.tensor(tokenized_examples['input_ids'][i].unsqueeze(dim=0)).to('cuda'),
                    'attention_mask' : torch.tensor(tokenized_examples['attention_mask'][i].unsqueeze(dim=0)).to('cuda'),
                    'token_type_ids' : torch.tensor(tokenized_examples['token_type_ids'][i].unsqueeze(dim=0)).to('cuda')
                }
                tmp_score = c_encoder(**c_input).to('cpu')
                score += tmp_score
            score = score / len(tokenized_examples['input_ids'])
            question_score.append(score)
        sort_result = torch.sort(torch.tensor(question_score), descending=True)
        scores, index_list = sort_result[0], sort_result[1]

        result_scores.append(scores.tolist())
        result_indices.append(index_list.tolist())        

<ipython-input-11-e8a615bf4e65>:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'input_ids' : torch.tensor(tokenized_examples['input_ids'][i].unsqueeze(dim=0)).to('cuda'),
<ipython-input-11-e8a615bf4e65>:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'attention_mask' : torch.tensor(tokenized_examples['attention_mask'][i].unsqueeze(dim=0)).to('cuda'),
<ipython-input-11-e8a615bf4e65>:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'token_type_ids' : torch.tensor(tokenized_examples['token_type_ids'][i].unsqueeze(dim=0)).to('cuda')


### 점수 비교

In [136]:
final_indices = []
for i in range(len(doc_indices)) :
    t_list = [doc_indices[i][result_indices[i][k]] for k in range(100)]
    final_indices.append(t_list)

In [127]:
# CE
total = []
for idx, example in enumerate(
        tqdm(dataset['validation'], desc="Dense retrieval: ")
    ):
        tmp = {
            # Query와 해당 id를 반환합니다.
            "question": example["question"],
            "id": example["id"],
            # Retrieve한 Passage의 id, context를 반환합니다.
            "context_id": final_indices[idx],
            "context": " ".join(  # 기존에는 ' '.join()
                [corpus[pid] for pid in final_indices[idx]]
            ),
        }
        if "context" in example.keys() and "answers" in example.keys():
            # validation 데이터를 사용하면 ground_truth context와 answer도 반환합니다.
            tmp["original_context"] = example["context"]
            tmp["answers"] = example["answers"]
        total.append(tmp)

CE_data = pd.DataFrame(total)

In [128]:
correct_length = []
for i in range(len(CE_data)) :
    if CE_data['original_context'][i] in CE_data['context'][i] :
        correct_length.append(i)
print(len(correct_length) / len(dataset['validation']))

0.8666666666666667


In [137]:
elastic_indices = []
for i in range(len(doc_indices)) :
    t_list = [doc_indices[i][k] for k in range(100)]
    elastic_indices.append(t_list)

In [130]:
# Elastic
total = []
for idx, example in enumerate(
        tqdm(dataset['validation'], desc="Dense retrieval: ")
    ):
        tmp = {
            # Query와 해당 id를 반환합니다.
            "question": example["question"],
            "id": example["id"],
            # Retrieve한 Passage의 id, context를 반환합니다.
            "context_id": elastic_indices[idx],
            "context": " ".join(  # 기존에는 ' '.join()
                [corpus[pid] for pid in elastic_indices[idx]]
            ),
        }
        if "context" in example.keys() and "answers" in example.keys():
            # validation 데이터를 사용하면 ground_truth context와 answer도 반환합니다.
            tmp["original_context"] = example["context"]
            tmp["answers"] = example["answers"]
        total.append(tmp)

elasitc_data = pd.DataFrame(total)

In [131]:
correct_length = []
for i in range(len(elasitc_data)) :
    if elasitc_data['original_context'][i] in elasitc_data['context'][i] :
        correct_length.append(i)
print(len(correct_length) / len(dataset['validation']))

0.8541666666666666


In [25]:
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
scaler = MinMaxScaler()
scale_ce = []
scale_elastic = []
for ce, elastic in tqdm(zip(result_scores, doc_scores)) :
    temp_ce = np.array(ce).reshape(-1, 1)
    temp_el = np.array(elastic).reshape(-1, 1)
    scale_ce.append(scaler.fit_transform(temp_ce).reshape(-1).tolist())
    scale_elastic.append(scaler.fit_transform(temp_el).reshape(-1).tolist())

In [138]:
real_scores = []
real_indices = []
for i in range(len(scale_ce)) :
    doc = {}
    for j in range(len(scale_ce[0])) :
        doc[final_indices[i][j]] = scale_ce[i][j]
    
    for j in range(len(scale_elastic[0])) :
        if doc_indices[i][j] in doc.keys() :
            doc[doc_indices[i][j]] += scale_elastic[i][j]
    doc = dict(sorted(doc.items(), key = lambda x: x[1], reverse = True))
    real_scores.append(list(doc.values()))
    real_indices.append(list(doc.keys()))

In [187]:
real_scores = []
real_indices = []
for i in range(len(scale_ce)) :
    doc = {}
    for j in range(len(scale_ce[0])) :
        doc[final_indices[i][j]] = scale_ce[i][j] * 3 # cross 1.1배
    
    for j in range(len(scale_elastic[0])) :
        if doc_indices[i][j] in doc.keys() :
            doc[doc_indices[i][j]] += scale_elastic[i][j]
    doc = dict(sorted(doc.items(), key = lambda x: x[1], reverse = True))
    real_scores.append(list(doc.values()))
    real_indices.append(list(doc.keys()))

In [188]:
i = 0
print('elastic indices:', doc_indices[i][:10])
print('cross encoder:', final_indices[i][:10])
print('ensemble:', real_indices[i][:10])


elastic indices: [17913, 17914, 9027, 4473, 49024, 49398, 46605, 15066, 20072, 41816]
cross encoder: [9027, 4473, 22077, 38855, 24768, 20072, 15066, 15634, 1277, 5831]
ensemble: [9027, 4473, 20072, 15066, 22077, 38855, 2321, 6875, 9497, 1277]


In [189]:
 [9027, 4473, 17914, 20072, 17913, 15066, 2321, 6875, 49398, 9497]

[9027, 4473, 17914, 20072, 17913, 15066, 2321, 6875, 49398, 9497]

In [190]:
the_end_indices = []
for i in range(len(real_indices)) :
    t_list = [real_indices[i][k] for k in range(5)]
    the_end_indices.append(t_list)

In [191]:
final = []
for idx, example in enumerate(
        tqdm(dataset['validation'], desc="Dense retrieval: ")
    ):
        tmp = {
            # Query와 해당 id를 반환합니다.
            "question": example["question"],
            "id": example["id"],
            # Retrieve한 Passage의 id, context를 반환합니다.
            "context_id": the_end_indices[idx],
            "context": " ".join(  # 기존에는 ' '.join()
                [corpus[pid] for pid in the_end_indices[idx]]
            ),
        }
        if "context" in example.keys() and "answers" in example.keys():
            # validation 데이터를 사용하면 ground_truth context와 answer도 반환합니다.
            tmp["original_context"] = example["context"]
            tmp["answers"] = example["answers"]
        final.append(tmp)

cqas_50 = pd.DataFrame(final)

In [192]:
correct_length = []
for i in range(len(cqas_50)) :
    if cqas_50['original_context'][i] in cqas_50['context'][i] :
        correct_length.append(i)
print(len(correct_length) / len(dataset['validation']))

0.9041666666666667


In [ ]:
0.8791666666666667

In [193]:
cqas_50.to_csv('retrieval_elastic_cross_ensemble_5_one_on_three.csv', index = False)

In [194]:
cqas_50

,question,id,context_id,context,original_context,answers
0,처음으로 부실 경영인에 대한 보상 선고를 받은 회사는?,mrc-0-003264,"[9027, 4473, 20072, 15066, 22077]","순천여자고등학교 졸업, 1973년 이화여자대학교를 졸업하고 1975년 제17회 사법...","순천여자고등학교 졸업, 1973년 이화여자대학교를 졸업하고 1975년 제17회 사법...","{'answer_start': [284], 'text': ['한보철강']}"
1,스카버러 남쪽과 코보콘그 마을의 철도 노선이 처음 연장된 연도는?,mrc-0-004762,"[51765, 21916, 23373, 37730, 23603]",요크 카운티 동쪽에 처음으로 여객 열차 운행이 시작한 시점은 1868년 토론토 & ...,요크 카운티 동쪽에 처음으로 여객 열차 운행이 시작한 시점은 1868년 토론토 & ...,"{'answer_start': [146], 'text': ['1871년']}"
2,촌락에서 운영 위원 후보자 이름을 쓰기위해 사용된 것은?,mrc-1-001810,"[15694, 746, 5300, 28167, 19853]","촐라 정부\n 촐라의 정부 체제는 전제군주제였으며,2001 촐라의 군주는 절대적인 ...","촐라 정부\n 촐라의 정부 체제는 전제군주제였으며,2001 촐라의 군주는 절대적인 ...","{'answer_start': [517], 'text': ['나뭇잎']}"
3,로타이르가 백조를 구하기 위해 사용한 것은?,mrc-1-000219,"[59536, 59537, 59533, 59534, 59527]",프랑스의 십자군 무훈시는 1099년 예루살렘 왕국의 통치자가 된 고드프루아 드 부용...,프랑스의 십자군 무훈시는 1099년 예루살렘 왕국의 통치자가 된 고드프루아 드 부용...,"{'answer_start': [1109], 'text': ['금대야']}"
4,의견을 자유롭게 나누는 것은 조직 내 어떤 관계에서 가능한가?,mrc-1-000285,"[32991, 46612, 12709, 20645, 46412]",탈관료제화는 현대사회에서 관료제 성격이 약화되는 현상이다. 현대사회에서 관료제는 약...,탈관료제화는 현대사회에서 관료제 성격이 약화되는 현상이다. 현대사회에서 관료제는 약...,"{'answer_start': [386], 'text': ['수평적 관계']}"
...,...,...,...,...,...,...
235,전단이 연나라와의 전쟁에서 승리했을 당시 제나라의 왕은 누구인가?,mrc-0-000484,"[53270, 53263, 52028, 53265, 53267]","기원전 265년, 전단은 조나라의 군사를 거느리고 연나라를 공격하여 중양(中陽)을 ...","연나라 군대의 사령관이 악의에서 기겁으로 교체되자, 전단은 스스로 신령의 계시를 받...","{'answer_start': [1084], 'text': ['제 양왕']}"
236,공놀이 경기장 중 일부는 어디에 위치하고 있나?,mrc-0-002095,"[12190, 36694, 7781, 3227, 11634]",현재 우리가 볼 수 있는 티칼의 모습은 펜실베이니아 대학교와 과테말라 정부의 협조 ...,현재 우리가 볼 수 있는 티칼의 모습은 펜실베이니아 대학교와 과테말라 정부의 협조 ...,"{'answer_start': [343], 'text': [''일곱 개의 신전 광장..."
237,창씨개명령의 시행일을 미루는 것을 수락한 인물은?,mrc-0-003083,"[772, 5326, 34671, 9029, 4475]",1940년 5월 1일 오전 창씨개명에 비협조적이라는 이유로 조선총독부 경무국에서 소...,1940년 5월 1일 오전 창씨개명에 비협조적이라는 이유로 조선총독부 경무국에서 소...,"{'answer_start': [247], 'text': ['미나미 지로']}"
238,망코 잉카가 쿠스코를 되찾기 위해 마련한 군사는 총 몇 명인가?,mrc-0-002978,"[48575, 55432, 56329, 55430, 48576]",빌카밤바 지역은 파차쿠티 황제 때 부터 잉카 제국에 속해있던 지역이었다. 스페인 군...,빌카밤바 지역은 파차쿠티 황제 때 부터 잉카 제국에 속해있던 지역이었다. 스페인 군...,"{'answer_start': [563], 'text': ['200,000명']}"


In [59]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Tue Nov  2 10:29:59 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.80.02    Driver Version: 450.80.02    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  Off  | 00000000:00:05.0 Off |                  Off |
| N/

In [108]:
real_final = []
for ce, ela in zip(scale_ce, scale_elastic) :
    tmp = []
    for i in range(len(ce)) :
        tmp.append(1.1 * ce[i]+ela[i])
    real_final.append(tmp)

In [113]:
torch.sort(torch.tensor(real_final[0]), descending = True)

torch.return_types.sort(
values=tensor([2.1000, 2.0982, 1.8127, 1.8107, 1.6259, 1.5978, 1.4961, 1.4187, 1.3991,
        1.3954, 1.3876, 1.3740, 1.3413, 1.2848, 1.2575, 1.2426, 1.1784, 1.1174,
        1.0878, 1.0532, 1.0370, 0.9837, 0.9260, 0.8683, 0.8343, 0.7762, 0.7731,
        0.7653, 0.7610, 0.7204, 0.7159, 0.7084, 0.7009, 0.6846, 0.6565, 0.6267,
        0.6198, 0.5877, 0.5691, 0.5605, 0.5348, 0.5290, 0.5242, 0.5179, 0.5124,
        0.4975, 0.4758, 0.4556, 0.4359, 0.4260, 0.4100, 0.3862, 0.3787, 0.3689,
        0.3548, 0.3487, 0.3296, 0.3193, 0.2950, 0.2881, 0.2771, 0.2761, 0.2637,
        0.2528, 0.2493, 0.2343, 0.2275, 0.2177, 0.2175, 0.2107, 0.2016, 0.1967,
        0.1899, 0.1865, 0.1862, 0.1691, 0.1612, 0.1585, 0.1397, 0.1347, 0.1314,
        0.1302, 0.1278, 0.1180, 0.1147, 0.1051, 0.1000, 0.0915, 0.0873, 0.0757,
        0.0710, 0.0702, 0.0687, 0.0664, 0.0441, 0.0359, 0.0274, 0.0193, 0.0134,
        0.0000]),
indices=tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13

## Test

In [299]:
tttt = pd.read_csv('/opt/ml/custom/top100_wikipedia.csv')

In [318]:
for i in range(len(tttt)) :
    if len(list(set(eval(tttt['document_id'][i])))) != 100 :
        print('ㅈ망')
        break

In [208]:
dataset = load_from_disk('/opt/ml/data/test_dataset')

In [313]:
print(len(set(eval(data['document_id'][6]))))

91


In [233]:
data = pd.read_csv('/opt/ml/custom/test_elastic_top100.csv')

doc_indices = []
doc_scores = []
for i in range(len(data)) :
    tmp_indices = eval(data['document_id'][i])
    tmp_scores = eval(data['score'][i])
    doc_indices.append(tmp_indices)
    doc_scores.append(tmp_scores)

In [251]:
# test에 대해서만 실행
for i in tqdm(range(len(doc_indices))) :
    for j in range(len(doc_indices[i])) :
        doc_indices[i][j] = int(doc_indices[i][j])

In [211]:
with open('/opt/ml/data/wikipedia_documents.json', "r", encoding="utf-8") as f:
    wiki = json.load(f)

corpus = []
for v in wiki.values() :
    corpus.append(v['text'])

In [212]:
class BertEncoder(BertPreTrainedModel):
    def __init__(self, config):
        super(BertEncoder, self).__init__(config)

        self.bert = BertModel(config)
        self.init_weights()
        classifier_dropout=(
            config.classifier_dropout if config.classifier_dropout is not None else config.hidden_dropout_prob
        )
        self.dropout = torch.nn.Dropout(classifier_dropout)
        self.linear = torch.nn.Linear(config.hidden_size, 1)
      
    def forward(
            self,
            input_ids, 
            attention_mask=None,
            token_type_ids=None
        ): 

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        
        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output)
        output = self.linear(pooled_output)
        return output

In [213]:
c_encoder = torch.load('/opt/ml/custom/c_encoder_e40_b16.pt')
model_checkpoint = "klue/bert-base"
# 혹시 위에서 사용한 encoder가 있다면 주석처리 후 진행해주세요 (CUDA ...)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
question_data = dataset['validation']['question']
with torch.no_grad() : 
    c_encoder.eval()

    result_scores = []
    result_indices = []
    for i in tqdm(range(len(question_data))) :
        question = question_data[i]
        question_score = []
        for indice in tqdm(doc_indices[i]) :
            passage = corpus[int(indice)]
            tokenized_examples = tokenizer(
                question,
                passage,
                truncation="only_second",
                max_length=512,
                stride=128,
                return_overflowing_tokens=True,
                return_offsets_mapping=True,
                #return_token_type_ids=False,  # roberta모델을 사용할 경우 False, bert를 사용할 경우 True로 표기해야합니다.
                padding="max_length",
                return_tensors='pt'
            )
            score = 0
            for i in range(len(tokenized_examples['input_ids'])) :
                c_input = {
                    'input_ids' : torch.tensor(tokenized_examples['input_ids'][i].unsqueeze(dim=0)).to('cuda'),
                    'attention_mask' : torch.tensor(tokenized_examples['attention_mask'][i].unsqueeze(dim=0)).to('cuda'),
                    'token_type_ids' : torch.tensor(tokenized_examples['token_type_ids'][i].unsqueeze(dim=0)).to('cuda')
                }
                tmp_score = c_encoder(**c_input).to('cpu')
                score += tmp_score
            score = score / len(tokenized_examples['input_ids'])
            question_score.append(score)
        sort_result = torch.sort(torch.tensor(question_score), descending=True)
        scores, index_list = sort_result[0], sort_result[1]

        result_scores.append(scores.tolist())
        result_indices.append(index_list.tolist())        

### 점수 비교

In [269]:
final_indices = []
for i in range(len(doc_indices)) :
    t_list = [doc_indices[i][result_indices[i][k]] for k in range(100)]
    final_indices.append(t_list)

elastic_indices = []
for i in range(len(doc_indices)) :
    t_list = [doc_indices[i][k] for k in range(100)]
    elastic_indices.append(t_list)

In [253]:
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
scaler = MinMaxScaler()
scale_ce = []
scale_elastic = []
for ce, elastic in tqdm(zip(result_scores, doc_scores)) :
    temp_ce = np.array(ce).reshape(-1, 1)
    temp_el = np.array(elastic).reshape(-1, 1)
    scale_ce.append(scaler.fit_transform(temp_ce).reshape(-1).tolist())
    scale_elastic.append(scaler.fit_transform(temp_el).reshape(-1).tolist())

In [328]:
real_scores = []
real_indices = []
for i in range(len(scale_ce)) :
    doc = {}
    for j in range(len(scale_ce[0])) :
        doc[final_indices[i][j]] = scale_ce[i][j] * 3
    
    for j in range(len(scale_elastic[0])) :
        if doc_indices[i][j] in doc.keys() :
            doc[doc_indices[i][j]] += scale_elastic[i][j]
    doc = dict(sorted(doc.items(), key = lambda x: x[1], reverse = True))
    real_scores.append(list(doc.values()))
    real_indices.append(list(doc.keys()))

In [329]:
the_end_indices = []
for i in range(len(real_indices)) :
    t_list = [real_indices[i][k] for k in range(4)]
    the_end_indices.append(t_list)

In [330]:
final = []
for idx, example in enumerate(
        tqdm(dataset['validation'], desc="Dense retrieval: ")
    ):
        tmp = {
            # Query와 해당 id를 반환합니다.
            "question": example["question"],
            "id": example["id"],
            # Retrieve한 Passage의 id, context를 반환합니다.
            "context_id": the_end_indices[idx],
            "context": " ".join(  # 기존에는 ' '.join()
                [corpus[pid] for pid in the_end_indices[idx]]
            ),
        }
        if "context" in example.keys() and "answers" in example.keys():
            # validation 데이터를 사용하면 ground_truth context와 answer도 반환합니다.
            tmp["original_context"] = example["context"]
            tmp["answers"] = example["answers"]
        final.append(tmp)

cqas_50 = pd.DataFrame(final)    

In [323]:
i = 0
print('elastic indices:', doc_indices[i][:10])
print('cross encoder:', final_indices[i][:10])
print('ensemble:', real_indices[i][:10])

elastic indices: [43280, 47081, 35064, 24024, 42242, 10509, 9781, 19993, 34043, 29886]
cross encoder: [24024, 20956, 3674, 3674, 49784, 52764, 42242, 42244, 14130, 29886]
ensemble: [24024, 42242, 47081, 3674, 42244, 43280, 35064, 29886, 20956, 9781]


In [331]:
cqas_50.to_csv('test_retrieval_elastic_cross_ensemble_4_one_on_three.csv', index = False)